In [16]:
from joblib import load
import numpy as np
import pandas as pd

import onnxruntime as rt

In [17]:
fea_dict={
    'all':[
"fft_imag_3",
"fft_angle_3",
"alt_stderr_min_50",
"alt_rvalue_min_50",
"fft_real_2",
"fft_real_5",
"erbc_num10_focus2",
"fft_imag_6",
"lumpiness"
],


 'opi':[
"fft_imag_3",
"erbc_num10_focus4",
"alt_rvalue_min_50",
"fft_angle_3",
"erbc_num10_focus2",
"fft_real_5"
],

 'cns':[
"fft_imag_3",
"fft_angle_3",
"fft_real_5",
"minindex",
"lumpiness",
"fft_imag_6",
"alt_rvalue_min_50",
"fft_angle_2",
"erbc_num10_focus2",
"S_down",
"retindex",
"erbc_num10_focus4",
"mac",
"alt_stderr_min_50",
],

'other':[
  "fft_imag_3",
  "fft_angle_3",
  "alt_stderr_min_50",
  "fft_imag_6",
  "fft_real_2",
  "alt_rvalue_min_50",
  "fft_real_5",
  "minvalue",
  "V2",
  "F2",
  "retindex"
]
}


##### 多种方式

In [18]:
n='n.csv'
# p='all.csv'

def ooo(way,count_array):
    pred_onx = eval(way+"_mod").run(
        [eval(way+"_mod").get_outputs()[0].name],
        {eval(way+"_mod").get_inputs()[0].name: count_array.astype(np.float32)})[0]

    return pred_onx[0]


all_mod=rt.InferenceSession("./mod/all.onnx")
opi_mod=rt.InferenceSession("./mod/opi.onnx")
cns_mod=rt.InferenceSession("./mod/cns.onnx")
other_mod=rt.InferenceSession("./mod/other.onnx")


def read_data(tit):
    path = 'E:\\desktop\\data\\app\\223\\fea\\ok\\'
    #读取数据
    data = pd.read_csv(path+tit,encoding='gbk')
    data = data.set_index('name')
    return data


fea_data=read_data(n)

result={'name':list(fea_data.index),'all':[],'opi':[],'cns':[],'other':[]}
for way in ('all','opi','cns','other'):
    count_array = np.array([[fea_data[fea] for fea in fea_dict[way]]])

    
    print(count_array)
    y_pred=ooo(way,count_array)
    result[way]=list(y_pred)

result=pd.DataFrame(result)
result=result.set_index('name',drop=True)

youla={"(opi or cns or other) and all":[],"(opi or cns) and all":[],"四选二":[]}
for i in range(len(result)):
    row=list(result.iloc[i])
    if sum(row[:3])>=1 and row[3]==1:
        youla["(opi or cns or other) and all"].append(1)
    else:
        youla["(opi or cns or other) and all"].append(0)

    if sum(row[:2])>=1 and row[3]==1:
        youla["(opi or cns) and all"].append(1)
    else:
        youla["(opi or cns) and all"].append(0)

    if sum(row)>=2:
        youla["四选二"].append(1)
    else:
        youla["四选二"].append(0)

youla=pd.DataFrame(youla,index=result.index)
result=pd.concat([result,youla],axis=1)
psum=len(result)
for key in result.columns:
    p=sum(result[key])
    print(key,p,'/',psum,p/psum)

[[[0.60218831 0.61401198 0.69339274 ... 0.68372391 0.81089798 0.66955682]
  [0.3046255  0.5101851  0.51708475 ... 0.34901531 0.56711704 0.58254619]
  [0.58555531 0.47470808 0.59262377 ... 0.29052595 0.68255025 0.62204736]
  ...
  [1.         0.4631438  0.58812279 ... 0.52747149 0.44466781 0.51040121]
  [0.45170124 0.53230145 0.42142783 ... 0.76652468 0.69528821 0.40661598]
  [0.6973058  0.10239436 0.32326926 ... 0.03359206 0.39150007 0.23966459]]]


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid rank for input: float_input Got: 3 Expected: 2 Please fix either the inputs or the model.